In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge ,BayesianRidge ,LinearRegression,Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from math import radians, cos, sin, asin, sqrt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading data

In [ ]:
s1 = pd.read_csv("/content/drive/MyDrive/s1.csv")
df2001 = pd.read_csv("/content/drive/MyDrive/df2001.csv")
df2003 = pd.read_csv("/content/drive/MyDrive/df2003.csv")
df2005 = pd.read_csv("/content/drive/MyDrive/df2005.csv")
df2007 = pd.read_csv("/content/drive/MyDrive/df2007.csv")
df2009 = pd.read_csv("/content/drive/MyDrive/df2009.csv")
df2011 = pd.read_csv("/content/drive/MyDrive/df2011.csv")
df2013 = pd.read_csv("/content/drive/MyDrive/df2013.csv")
df2015 = pd.read_csv("/content/drive/MyDrive/df2015.csv")
df2017 = pd.read_csv("/content/drive/MyDrive/df2017.csv")
df2019 = pd.read_csv("/content/drive/MyDrive/df2019.csv")
df2021 = pd.read_csv("/content/drive/MyDrive/df2021.csv")

## Droping cols and OneHotEncoding aka set dummy variable

In [ ]:
def preprocessing(s1):
  s1 = s1.drop(['lease_commence_date'], axis = 1)
  s1 = s1.drop(['Unnamed: 0'],axis = 1)
  s1 = s1.drop(['address'],axis = 1)
  onehot = OneHotEncoder()
  town_df = pd.DataFrame(onehot.fit_transform(s1[['town']]).toarray())
  town_df.columns = onehot.get_feature_names_out(['town'])
  flat_model_df = pd.DataFrame(onehot.fit_transform(s1[['flat_model']]).toarray())
  flat_model_df.columns = onehot.get_feature_names_out(['flat_model'])
  flat_type_df = pd.DataFrame(onehot.fit_transform(s1[['flat_type']]).toarray())
  flat_type_df.columns = onehot.get_feature_names_out(['flat_type'])
  s1 = s1.join([town_df, flat_model_df, flat_type_df])
  s1 = s1.drop(['town','flat_type','flat_model'], axis = 1)
  return s1


In [ ]:
s1 =preprocessing(s1)
df2001 = preprocessing(df2001)
df2003 = preprocessing(df2003)
df2005 = preprocessing(df2005)
df2007 = preprocessing(df2007)
df2009 = preprocessing(df2009)
df2011 = preprocessing(df2011)
df2013 = preprocessing(df2013)
df2015 = preprocessing(df2015)
df2017 = preprocessing(df2017)
df2019 = preprocessing(df2019)
df2021 = preprocessing(df2021)

In [ ]:
add_flat = np.zeros(11553)
df2021["flat_model_Premium Maisonette"] = add_flat

## DecisionTreeRegressor

In [ ]:
def custom_DT(df,s1):
  test  = s1.loc[s1['k_fold'] == 1]
  test  =test.drop(['k_fold'],axis = 1)
  y_test   = test['resale_price'].tolist()
  test  = test.drop(['resale_price'],axis=1)
  X_test = test.to_numpy()
  k_error_list = []
  S1_error_list = []
  for x in range(1,11):
    Validation  = df.loc[df['k_fold'] == x]
    train =df.loc[df['k_fold'] != x]
    train =train.drop(['k_fold'],axis = 1)
    Validation =Validation.drop(['k_fold'],axis = 1)
    y_train = train['resale_price'].tolist()
    y_Validation  = Validation ['resale_price'].tolist()
    train = train.drop(['resale_price'],axis=1)
    Validation  = Validation.drop(['resale_price'],axis=1)
    X_train = train.to_numpy()
    X_Validation  = Validation.to_numpy()
# Modelling 
    model = DecisionTreeRegressor(max_depth=4)
    model.fit(X_train,y_train)
    y_validation_pred = model.predict(X_Validation)
    sse = sum((y_Validation- y_validation_pred)**2)
    k_error_list.append(sse)
    

    y_pred_test = y_pred = model.predict(X_test)
    test_sse = sum((y_pred_test- y_test)**2)
    S1_error_list.append(test_sse)
  return(k_error_list,S1_error_list)

In [ ]:
data_list = [df2001,df2003,df2005,df2007,df2009,df2011,df2013,df2015,df2017,df2019,df2021]
summary_DT_p1 = {}
for i in range(0,11):
  summary_DT_p1[str(i)] = custom_DT(data_list[i],s1)
summary_DT_p1

{'0': ([112374284129009.02,
   113232956732096.06,
   116416763094782.83,
   113951109678108.47,
   114558305689941.98,
   114438341918031.84,
   109654528654441.75,
   114019038424579.03,
   113551873892275.28,
   113637641186526.2],
  [19609923217777.363,
   19601846694483.86,
   19603755596228.805,
   19774114955319.08,
   19589930410215.285,
   19614211739396.695,
   19569802547711.758,
   19569490909963.258,
   19595173115698.246,
   19578574515995.664]),
 '1': ([106946241355330.7,
   103452706722141.56,
   104719245031546.78,
   103476840616374.53,
   104462993411588.12,
   107119391243794.78,
   105440208187743.14,
   106705532429294.48,
   105812491141934.1,
   105058486447362.84],
  [19386568980040.984,
   19348049866813.598,
   19337990592612.56,
   19366489496713.023,
   19357661191975.51,
   19601676501540.465,
   19571905168156.406,
   19398547337811.418,
   19368995610360.61,
   19358898526897.766]),
 '10': ([6936339062710.46,
   7520443873906.822,
   7823703982477.712,
 

##Ridge regression

In [ ]:
# Model fitting function for ridge regression
def custom_Ridge(df,s1):
  test  = s1.loc[s1['k_fold'] == 1]
  test  =test.drop(['k_fold'],axis = 1)
  y_test   = test['resale_price'].tolist()
  test  = test.drop(['resale_price'],axis=1)
  X_test = test.to_numpy()
  k_error_list = []
  S1_error_list = []
  for x in range(1,11):
    Validation  = df.loc[df['k_fold'] == x]
    train =df.loc[df['k_fold'] != x]
    train =train.drop(['k_fold'],axis = 1)
    Validation =Validation.drop(['k_fold'],axis = 1)
    y_train = train['resale_price'].tolist()
    y_Validation  = Validation ['resale_price'].tolist()
    train = train.drop(['resale_price'],axis=1)
    Validation  = Validation.drop(['resale_price'],axis=1)
    X_train = train.to_numpy()
    X_Validation  = Validation.to_numpy()
# Modelling 
    model = Ridge(alpha=5)
    model.fit(X_train,y_train)
    y_validation_pred = model.predict(X_Validation)
    sse = sum((y_Validation- y_validation_pred)**2)
    k_error_list.append(sse)
    

    y_pred_test = y_pred = model.predict(X_test)
    test_sse = sum((y_pred_test- y_test)**2)
    S1_error_list.append(test_sse)
  return(k_error_list,S1_error_list) 

In [ ]:
data_list = [df2001,df2003,df2005,df2007,df2009,df2011,df2013,df2015,df2017,df2019,df2021]
summary_ridge_p1 = {}
for i in range(0,11):
  summary_ridge_p1[str(i)] = custom_Ridge(data_list[i],s1)
summary_ridge_p1

{'0': ([49269321861755.56,
   50888485268818.805,
   49900381336117.12,
   49009652921617.94,
   49412743773809.23,
   50470619641146.016,
   47831632730519.37,
   48898522359000.19,
   49247833074226.58,
   50529201756605.016],
  [5942276641476.699,
   5929493538360.131,
   5932938283289.452,
   5930033798370.583,
   5943066419849.572,
   5941482270429.247,
   5932254418749.7705,
   5936006124613.4375,
   5921757379641.519,
   5945954136165.12]),
 '1': ([45383024504903.18,
   46018813972141.086,
   44276530330052.98,
   45618395585880.72,
   45096934187235.53,
   45468508437826.695,
   45411305414172.375,
   46343940001269.37,
   45468974643021.22,
   44541188475153.125],
  [5835388381610.52,
   5848028028541.268,
   5844463537397.851,
   5848740645890.673,
   5834082846640.842,
   5836960529724.1045,
   5838318675047.465,
   5845882427949.486,
   5843119500903.377,
   5836269625807.814]),
 '10': ([2847953903540.9985,
   3183968081358.5234,
   3275144775616.599,
   3165126540810.1064,

#K Nearest Neighbours

In [ ]:
#K Nearest Neighbours 
def custom_knn(df,s1):
  test  = s1.loc[s1['k_fold'] == 1]
  test  =test.drop(['k_fold'],axis = 1)
  y_test   = test['resale_price'].tolist()
  test  = test.drop(['resale_price'],axis=1)
  X_test = test.to_numpy()
  k_error_list = []
  S1_error_list = []
  for x in range(1,11):
    Validation  = df.loc[df['k_fold'] == x]
    train =df.loc[df['k_fold'] != x]
    train =train.drop(['k_fold'],axis = 1)
    Validation =Validation.drop(['k_fold'],axis = 1)
    y_train = train['resale_price'].tolist()
    y_Validation  = Validation ['resale_price'].tolist()
    train = train.drop(['resale_price'],axis=1)
    Validation  = Validation.drop(['resale_price'],axis=1)
    X_train = train.to_numpy()
    X_Validation  = Validation.to_numpy()
    
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_validation_pred = knn.predict(X_Validation)
    sse = sum((y_Validation- y_validation_pred)**2)
    
    k_error_list.append(sse)

    y_pred_test = y_pred = knn.predict(X_test)
    test_sse = sum((y_pred_test - y_test)**2)
    
    S1_error_list.append(test_sse)
  return(k_error_list,S1_error_list)


In [ ]:
data_list = [df2001,df2003,df2005,df2007,df2009,df2011,df2013,df2015,df2017,df2019,df2021]
summary_knn = {}
for i in range(0,11):
  summary_knn[str(i)] = custom_knn(data_list[i],s1)
summary_knn


# **Lasso regression**

In [ ]:
# Data list 
# Note that for df2021 there is only 47 columns while for the rest is 48, 
# which is due to the lack of a flat_type flat_model_Premium Maisonette, to
# remedy this just add an additional column to df2021
add_flat = np.zeros(11553)
df2021["flat_model_Premium Maisonette"] = add_flat
data_list = [df2001,df2003,df2005,df2007,df2009,df2011,df2013,df2015,df2017,df2019,df2021]


In [ ]:
# Model fitting function for lasso regression
def custom_Lasso(df,s1):
  test  = s1.loc[s1['k_fold'] == 1]
  test  =test.drop(['k_fold'],axis = 1)
  y_test   = test['resale_price'].tolist()
  test  = test.drop(['resale_price'],axis=1)
  X_test = test.to_numpy()
  k_error_list = []
  S1_error_list = []
  for x in range(1,11):
    Validation  = df.loc[df['k_fold'] == x]
    train =df.loc[df['k_fold'] != x]
    train =train.drop(['k_fold'],axis = 1)
    Validation =Validation.drop(['k_fold'],axis = 1)
    y_train = train['resale_price'].tolist()
    y_Validation  = Validation ['resale_price'].tolist()
    train = train.drop(['resale_price'],axis=1)
    Validation  = Validation.drop(['resale_price'],axis=1)
    X_train = train.to_numpy()
    X_Validation  = Validation.to_numpy()
# Modelling 
    model = Lasso(max_iter=2000, tol = 0.1)
    model.fit(X_train,y_train)
    y_validation_pred = model.predict(X_Validation)
    val_sse = sum((y_Validation- y_validation_pred)**2)
    k_error_list.append(val_sse)
    

    y_pred_test = y_pred = model.predict(X_test)
    test_sse = sum((y_pred_test - y_test)**2)
    S1_error_list.append(test_sse)
  return(k_error_list,S1_error_list)


In [ ]:
summary_lasso_p1 = {}
for i in range(0,11):
  summary_lasso_p1[str(i)] = custom_Lasso(data_list[i],s1)

In [ ]:
summary_lasso_p1